In [ ]:
from utils import IOU_vectorized
import numpy as np


In [ ]:

def average_precision(test_loader, predicted_boxes, predicted_labels, confidence_scores, clas):
    num_pred_of_clas = np.sum(predicted_labels == clas)
    all_iou_scores = np.zeros(num_pred_of_clas)
    all_confidence_scores = np.zeros(num_pred_of_clas)
    cur_idx = 0
    num_gt_of_clas = 0

    for i, batch in enumerate(test_loader):
        gt_labels = batch['labels'][0]
        gt_boxes = batch['bboxes'][0]

        pred_boxes = predicted_boxes[i]
        pred_labels = predicted_labels[i]
        c_scores = confidence_scores[i]
        
        num_gt_of_clas += np.sum(gt_labels==clas)
        gt_idx = np.argwhere(gt_labels==clas)
        pred_idx = np.argwhere(pred_labels==clas)
        
        iou_scores = IOU_vectorized(pred_boxes[pred_idx], gt_boxes[gt_idx])
        max_iou_scores = np.max(iou_scores, axis=0)

        num_preds = max_iou_scores.shape[0]
        all_iou_scores[cur_idx:(cur_idx + num_preds)] = max_iou_scores
        all_confidence_scores[cur_idx:(cur_idx + num_preds)] = c_scores

    order = all_confidence_scores.argsort()[::-1]
    sorted_iou_scores = all_iou_scores[order]

    tp = np.cumsum(sorted_iou_scores > 0.5)
    fp = np.sumsum(sorted_iou_scores < 0.5)
    precision = tp / (fp + tp)
    recall = tp / num_gt_of_clas

    average_precision = 0
    prev_recall = 0
    for j in range(len(recall)-1):
        if precision[j]>precision[j+1]:
            average_precision+=(recall[j] - prev_recall)*precision[j]
            prev_recall = recall[j]
    
    if(len(recall)!=0):
        average_precision += precision[len(recall)-1]*(recall[len(recall)-1]- prev_recall)
    else:
        average_precision = 0
    return average_precision


